In [1]:
with open("the-verdict.txt","r", encoding = "utf-8") as f:
    raw_text=f.read()
    
print("Number of characters", len(raw_text))
print(raw_text[:99])

Number of characters 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


# Creating Tokenizer 

In [2]:
import re
text = "Hello, world. --Hi I'm LLM, Is a test? If yes: I'm going to smash it!"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
print(result)
result =[item.strip() for item in result if item.strip()]
print('\n',result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', '', '--', 'Hi', ' ', 'I', "'", 'm', ' ', 'LLM', ',', '', ' ', 'Is', ' ', 'a', ' ', 'test', '?', '', ' ', 'If', ' ', 'yes', ':', '', ' ', 'I', "'", 'm', ' ', 'going', ' ', 'to', ' ', 'smash', ' ', 'it', '!', '']

 ['Hello', ',', 'world', '.', '--', 'Hi', 'I', "'", 'm', 'LLM', ',', 'Is', 'a', 'test', '?', 'If', 'yes', ':', 'I', "'", 'm', 'going', 'to', 'smash', 'it', '!']


In [10]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed =[item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:50])
print('\n',len(preprocessed))

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of', 'his', 'glory', ',', 'he', 'had', 'dropped', 'his', 'painting', ',', 'married', 'a', 'rich', 'widow', ',', 'and', 'established', 'himself']

 4690


# Creating Token IDs

In [12]:
words = sorted(set(preprocessed))

print(len(words))

1130


In [13]:
vocab = {token:integer for integer,token in enumerate(words)}

In [18]:
for i, (token, idx) in enumerate(vocab.items()):
    if i >= 20:
        break
    print(f"{i:2d}: {token}")


 0: !
 1: "
 2: '
 3: (
 4: )
 5: ,
 6: --
 7: .
 8: :
 9: ;
10: ?
11: A
12: Ah
13: Among
14: And
15: Are
16: Arrt
17: As
18: At
19: Be


In [83]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])

        # Remove space before punctuation
        text = re.sub(r"\s+([,.?;:!()\"'])", r"\1", text)

        # Remove space only after opening quotes — not closing
        text = re.sub(r'(^|[\s(\["])\s+', r'\1', text)

        # Fix -- spacing
        text = re.sub(r"\s+--\s+", "--", text)

        # Fix common contractions
        text = re.sub(r"'\s+s\b", "'s", text)
        text = re.sub(r"'\s+t\b", "'t", text)
        text = re.sub(r"'\s+re\b", "'re", text)
        text = re.sub(r"'\s+ve\b", "'ve", text)
        text = re.sub(r"'\s+d\b", "'d", text)
        text = re.sub(r"'\s+ll\b", "'ll", text)
        text = re.sub(r"'\s+m\b", "'m", text)

        # Add space after ?" or !" if followed by any letter
        text = re.sub(r'([,?.!])(")([A-Za-z])', r'\1\2 \3', text)

        return text

In [86]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"Why _has_ he chucked painting?" I asked abruptly."""
ids = tokenizer.encode(text)
print(ids)

[1, 111, 114, 529, 114, 533, 261, 748, 10, 1, 53, 179, 120, 7]


In [87]:
text = tokenizer.decode(ids)
print(text)

"Why _ has _ he chucked painting?" I asked abruptly.


# Adding Special tokens


In [91]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}
print(len(vocab))  
print(list(vocab.items())[-5:])

1132
[('younger', 1127), ('your', 1128), ('yourself', 1129), ('<|endoftext|>', 1130), ('<|unk|>', 1131)]


In [114]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])

        # Remove space before punctuation
        text = re.sub(r"\s+([,.?;:!()\"'])", r"\1", text)

        # Remove space only after opening quotes — not closing
        text = re.sub(r'(^|[\s(\["])\s+', r'\1', text)

        # Fix -- spacing
        text = re.sub(r"\s+--\s+", "--", text)

        # Fix common contractions
        text = re.sub(r"'\s+s\b", "'s", text)
        text = re.sub(r"'\s+t\b", "'t", text)
        text = re.sub(r"'\s+re\b", "'re", text)
        text = re.sub(r"'\s+ve\b", "'ve", text)
        text = re.sub(r"'\s+d\b", "'d", text)
        text = re.sub(r"'\s+ll\b", "'ll", text)
        text = re.sub(r"'\s+m\b", "'m", text)

        # Add space after ?" or ," etc. if followed by a word (any letter)
        text = re.sub(r'([,?.!])(")([A-Za-z])', r'\1\2 \3', text)

        return text

In [115]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "She raised her eyebrows with a hint of good-humoured surprise."

text = " <|endoftext|> ".join((text1,text2))
print(text)

Hello, do you like tea? <|endoftext|> She raised her eyebrows with a hint of good-humoured surprise.


In [116]:
tokenizer.encode(text)

[1131,
 5,
 355,
 1126,
 628,
 975,
 10,
 1130,
 88,
 816,
 539,
 416,
 1108,
 115,
 548,
 722,
 502,
 961,
 7]

In [117]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> She raised her eyebrows with a hint of good-humoured surprise.'

"Hello" was not vocab but SimpleTokenizerV2 handles it now

# Byte pair Encoding 


With chatgpt tiktoken

In [118]:
! pip3 install tiktoken

In [123]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))
tokenizer = tiktoken.get_encoding("gpt2")

tiktoken version: 0.9.0


In [124]:
text = ("Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace.")

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [126]:
text = tokenizer.decode(integers)

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


# input-target pair

In [127]:
encoded_text = tokenizer.encode(raw_text) #byte pair encoding
print(len(encoded_text))

5145


In [128]:
encoded_sample = encoded_text[50:]
print(len(encoded_sample))

5095


In [132]:
context_size = 4

x = encoded_sample[:context_size]
y = encoded_sample[1:context_size+1]
print(f'x:',x)
print(f'y:     ',y)

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [139]:
for i in range(1, context_size+1):
    context = encoded_sample[:i]
    desired = encoded_sample[i]
    print(tokenizer.decode(context), "-->",tokenizer.decode([desired]))

 and -->  established
 and established -->  himself
 and established himself -->  in
 and established himself in -->  a


# Data loader


In [140]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [141]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [142]:
import torch
print("PyTorch version:", torch.__version__)
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

PyTorch version: 2.6.0
[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [143]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [144]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
